In [2]:
def results(tp, fp, fn):
    if(tp == 0 and fp == 0):
        precision = 0
    else:
        precision = tp/(tp + fp)
    if(tp == 0 and fn == 0):
        recall = 0
    else:
        recall = tp/(tp + fn)
    if(precision + recall == 0):
        f1 = 0
    else:
        f1 = 2 * (precision * recall)/(precision + recall)
    print("tp: ", tp, " fp: ", fp, "fn: ", fn)
    print("Precision: ", round(precision, 2))
    print("Recall: ", round(recall, 2))
    print("F1: ", round(f1, 2))
    print()

In [3]:
import numpy as np

# Code borrowed from learnopencv: https://learnopencv.com/intersection-over-union-iou-in-object-detection-and-segmentation/
def get_iou(ground_truth, pred):
    # coordinates of the area of intersection.
    ix1 = np.maximum(ground_truth[0], pred[0])
    iy1 = np.maximum(ground_truth[1], pred[1])
    ix2 = np.minimum(ground_truth[2], pred[2])
    iy2 = np.minimum(ground_truth[3], pred[3])
     
    # Intersection height and width.
    i_height = np.maximum(iy2 - iy1 + 1, np.array(0.))
    i_width = np.maximum(ix2 - ix1 + 1, np.array(0.))
     
    area_of_intersection = i_height * i_width
     
    # Ground Truth dimensions.
    gt_height = ground_truth[3] - ground_truth[1] + 1
    gt_width = ground_truth[2] - ground_truth[0] + 1
     
    # Prediction dimensions.
    pd_height = pred[3] - pred[1] + 1
    pd_width = pred[2] - pred[0] + 1
     
    area_of_union = gt_height * gt_width + pd_height * pd_width - area_of_intersection
     
    iou = area_of_intersection / area_of_union
     
    return iou

In [4]:
import cv2
import os
import numpy as np
import pathlib
import dlib
from sklearn.metrics import mean_squared_error

# Load the face detection model
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('./shape_predictor_68_face_landmarks.dat/shape_predictor_68_face_landmarks.dat')
al = dlib.shape_predictor('./shape_predictor_68_face_landmarks.dat/shape_predictor_5_face_landmarks.dat')

cascade_path = pathlib.Path(cv2.__file__).parent.absolute() / "data/haarcascade_frontalface_alt.xml"
clf = cv2.CascadeClassifier(str(cascade_path))

font = cv2.FONT_HERSHEY_SIMPLEX
position = (10, 10) # (x, y) coordinates of the text position
position2 = (10, 20) # (x, y) coordinates of the text position

brightness = 200
contrast = 150

# Initialize counters for true positives, false positives, and false negatives
tp = 0
fp = 0
fn = 0
tp2 = 0
fp2 = 0
fn2 = 0
tp3 = 0
fp3 = 0
fn3 = 0
tp4 = 0
fp4 = 0
fn4 = 0

prec = ""
rec = ""
prec_max = 0
rec_max = 0


dir = "./smaller-data"
for filename in os.listdir(dir):
    filepath = (os.path.join(dir, filename))
    name = filename.split(".")
    name = name[0]

    videoFile = np.load(filepath)
    colorImages = videoFile['colorImages']
    boundingBox = videoFile['boundingBox']
    landmarks2D = videoFile['landmarks2D']

    # Initialize counters for true positives, false positives, and false negatives
    tpv = 0
    fpv = 0
    fnv = 0
    tp2v = 0
    fp2v = 0
    fn2v = 0
    tp3v = 0
    fp3v = 0
    fn3v = 0
    tp4v = 0
    fp4v = 0
    fn4v = 0

    height = colorImages.shape[0]
    width = colorImages.shape[1]
    size = (width, height)
    # out = cv2.VideoWriter(os.path.join('./videos',f'{name}.mp4'), cv2.VideoWriter_fourcc(*'h264'), 25, size, isColor=False)
    for i in range(colorImages.shape[-1]):
        viss = []
        img = np.ones((height, width, 3), np.uint8)
        img1 = colorImages[:, :, :, i]
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
        img2 = img1.copy()
        img3 = img1.copy()
        img4 = img1.copy()
        img5 = img1.copy()

        gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
        gray3 = cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY)
        gray4 = cv2.cvtColor(img4, cv2.COLOR_BGR2GRAY)

        # Enhance the brightness and contrast
        img3 = np.int16(img3)
        img3 = img3 * (contrast/10000+1) - contrast + brightness
        img3 = np.clip(img3, 0, 255)
        img3 = np.uint8(img3)
        img3 = cv2.GaussianBlur(img3, (7, 7), 0)

        img4 = np.int16(img4)
        img4 = img4 * (contrast/10000+1) - contrast + brightness
        img4 = np.clip(img4, 0, 255)
        img4 = np.uint8(img4)
        img4 = cv2.GaussianBlur(img4, (7, 7), 0)

        #Cascade Classifier
        faces1 = clf.detectMultiScale(
            gray,
            scaleFactor = 1.1,
            minNeighbors = 5,
            minSize = (30, 30),
            flags = cv2.CASCADE_SCALE_IMAGE
        )

        x, y, x, y = -1, -1, -1, -1
        for (x, y, w, h) in faces1:
            cv2.rectangle(img1, (x, y), (x + w, y + h), (255, 255, 0), 2)
            # detect landmarks (chatGPT helped me with this part)
            rect = dlib.rectangle(int(x), int(y), int(x + w), int(y + h))
            landmarks = predictor(gray, rect)
            pred_values1 = []
            for n in range(0, 68):
                p = landmarks.part(n).x
                q = landmarks.part(n).y
                pred_values1.append([p, q])
                cv2.circle(img1, (p, q), 1, (0, 0, 0), -1)
            break


        #Cascade Classifier Blurred
        faces3 = clf.detectMultiScale(
            gray3,
            scaleFactor = 1.1,
            minNeighbors = 5,
            minSize = (30, 30),
            flags = cv2.CASCADE_SCALE_IMAGE
        )

        xb, yb, xb, yb = -1, -1, -1, -1
        for (xb, yb, wb, hb) in faces3:
            cv2.rectangle(img3, (xb, yb), (xb + wb, yb + hb), (255, 255, 0), 2)
            # detect landmarks (chatGPT helped me with this part)
            rect = dlib.rectangle(int(xb), int(yb), int(xb + wb), int(yb + hb))
            landmarks = predictor(gray, rect)
            pred_values3 = []
            for n in range(0, 68):
                p = landmarks.part(n).x
                q = landmarks.part(n).y
                pred_values3.append([p, q])
                cv2.circle(img3, (p, q), 1, (0, 0, 0), -1)
            break


        x1, y1, x2, y2 = -1, -1, -1, -1
        #DLIB
        faces2 = detector(gray2)
        for face in faces2:
            landmarks = predictor(gray2, face)
            pred_values2 = []
            for n in range(0, 68):
                p = landmarks.part(n).x
                q = landmarks.part(n).y
                pred_values2.append([p, q])
                cv2.circle(img2, (p, q), 1, (0, 0, 0), 1)
            x1, y1 = face.left(), face.top()
            x2, y2 = face.right(), face.bottom()
            cv2.rectangle(img2, (x1, y1), (x2, y2), (0, 255, 0), 2)
            break


        x1b, y1b, x2b, y2b = -1, -1, -1, -1
        #DLIB blurred
        faces4 = detector(gray4)
        for face in faces4:
            landmarks = predictor(gray4, face)
            pred_values4 = []
            for n in range(0, 68):
                p = landmarks.part(n).x
                q = landmarks.part(n).y
                pred_values4.append([p, q])
                cv2.circle(img4, (p, q), 1, (0, 0, 0), 1)
            x1b, y1b = face.left(), face.top()
            x2b, y2b = face.right(), face.bottom()
            cv2.rectangle(img4, (x1b, y1b), (x2b, y2b), (0, 255, 0), 2)
            break
        

        bb = boundingBox[:, :, i]
        true_values = []
        for mark in landmarks2D[:, :, i]:
            true_values.append([int(mark[0]), int(mark[1])])
            cv2.circle(img, (int(mark[0]), int(mark[1])), 1, (255, 255, 255))

        bbox_true = [int(bb[0, 0]), int(bb[0, 1]), int(bb[3, 0]), int(bb[3, 1])]
        cv2.rectangle(img,
                        (int(bb[0, 0]), int(bb[0, 1])), 
                        (int(bb[3, 0]), int(bb[3, 1])),
                        (0, 0, 255), 1)
        
        #IOU
        if(x == -1):
            fn += 1
            fnv += 1
        else:
            iou = get_iou(bbox_true, [x, y, x + w, y + h])
            iou_str = f'CC IOU: {iou:.2f}'
            cv2.putText(img1, iou_str, position, font, 0.25, (0, 0, 0), 1, cv2.LINE_AA)
            # print('CC IOU: ', iou)

            mse = mean_squared_error(true_values, pred_values1)
            mse_str = f'CC MSE: {mse:.2f}'
            cv2.putText(img1, mse_str, position2, font, 0.25, (0, 0, 0), 1, cv2.LINE_AA)

            if(iou >= 0.5):
                tp += 1
                tpv += 1
            else:
                fp += 1
                fpv += 1

        if(xb == -1):
            fn3 += 1
            fn3v += 1
        else:
            iou = get_iou(bbox_true, [xb, yb, xb + wb, yb + hb])
            iou_str = f'CC IOU: {iou:.2f}'
            cv2.putText(img3, iou_str, position, font, 0.25, (0, 0, 0), 1, cv2.LINE_AA)
            # print('CC IOU: ', iou)

            mse = mean_squared_error(true_values, pred_values3)
            mse_str = f'CC MSE: {mse:.2f}'
            cv2.putText(img3, mse_str, position2, font, 0.25, (0, 0, 0), 1, cv2.LINE_AA)

            if(iou >= 0.5):
                tp3 += 1
                tp3v += 1
            else:
                fp3 += 1
                fp3v += 1


        if(x1 == -1):
            fn2 += 1
            fn2v += 1
        else:
            iou = get_iou(bbox_true, [x1, y1, x2, y2])
            iou_str = f'DLIB IOU: {iou:.2f}'
            cv2.putText(img2, iou_str, position, font, 0.25, (0, 0, 0), 1, cv2.LINE_AA)
            # print('CC IOU: ', iou)

            mse = mean_squared_error(true_values, pred_values2)
            mse_str = f'DLIB MSE: {mse:.2f}'
            cv2.putText(img2, mse_str, position2, font, 0.25, (0, 0, 0), 1, cv2.LINE_AA)

            if(iou >= 0.5):
                tp2 += 1
                tp2v += 1
            else:
                fp2 += 1
                fp2v += 1


        if(x1b == -1):
            fn4 += 1
            fn4v += 1
        else:
            iou = get_iou(bbox_true, [x1b, y1b, x2b, y2b])
            iou_str = f'DLIB IOU: {iou:.2f}'
            cv2.putText(img4, iou_str, position, font, 0.25, (0, 0, 0), 1, cv2.LINE_AA)
            # print('CC IOU: ', iou)

            mse = mean_squared_error(true_values, pred_values4)
            mse_str = f'DLIB MSE: {mse:.2f}'
            cv2.putText(img4, mse_str, position2, font, 0.25, (0, 0, 0), 1, cv2.LINE_AA)

            if(iou >= 0.5):
                tp4 += 1
                tp4v += 1
            else:
                fp4 += 1
                fp4v += 1

        combine = np.hstack((img, img1, img2, img3, img4))
        # out.write(img2)
        cv2.imshow('video', combine)
        c = cv2.waitKey(1)
        if c == 27:
            break
    # out.release()

    var = max(fnv, fn2v, fn3v, fn4v)
    if(var > rec_max):
        rec_max = var
        rec = name

    var = max(fpv, fp2v, fp3v, fp4v)
    if(var > prec_max):
        prec_max = var
        prec = name
    
    print(name ,":\n")
    results(tpv, fpv, fnv)
    results(tp2v, fp2v, fn2v)
    print("Modified:\n")
    results(tp3v, fp3v, fn3v)
    results(tp4v, fp4v, fn4v)
    cv2.destroyAllWindows()

print("Result for entire database:")
results(tp, fp, fn)
results(tp2, fp2, fn2)
print("Result for entire database (modified): ")
results(tp3, fp3, fn3)
results(tp4, fp4, fn4)
print("Lowest recall: ", prec)
print("Lowest precission: ", rec)

Karin_Viard_1 :

tp:  79  fp:  0 fn:  0
Precision:  1.0
Recall:  1.0
F1:  1.0

tp:  42  fp:  0 fn:  37
Precision:  1.0
Recall:  0.53
F1:  0.69

Modified:

tp:  79  fp:  0 fn:  0
Precision:  1.0
Recall:  1.0
F1:  1.0

tp:  42  fp:  0 fn:  37
Precision:  1.0
Recall:  0.53
F1:  0.69

Kate_Winslet_1 :

tp:  240  fp:  0 fn:  0
Precision:  1.0
Recall:  1.0
F1:  1.0

tp:  240  fp:  0 fn:  0
Precision:  1.0
Recall:  1.0
F1:  1.0

Modified:

tp:  240  fp:  0 fn:  0
Precision:  1.0
Recall:  1.0
F1:  1.0

tp:  240  fp:  0 fn:  0
Precision:  1.0
Recall:  1.0
F1:  1.0

Katharine_Hepburn_3 :

tp:  104  fp:  0 fn:  4
Precision:  1.0
Recall:  0.96
F1:  0.98

tp:  104  fp:  0 fn:  4
Precision:  1.0
Recall:  0.96
F1:  0.98

Modified:

tp:  104  fp:  0 fn:  4
Precision:  1.0
Recall:  0.96
F1:  0.98

tp:  104  fp:  0 fn:  4
Precision:  1.0
Recall:  0.96
F1:  0.98

Kathryn_Morris_5 :

tp:  43  fp:  0 fn:  0
Precision:  1.0
Recall:  1.0
F1:  1.0

tp:  43  fp:  0 fn:  0
Precision:  1.0
Recall:  1.0
F1:  1.0


KeyboardInterrupt: 

: 

In [ ]:
# Borrowed from https://github.com/Spidy20/Insta_flters_with_python/blob/master/put_hat%26glass.py
def put_mustas(mustas, img, x, y, w, h):
    face_width = w
    face_height = h
    mustas_width = face_width + 1
    mustas_height = int(0.25 * face_height) + 1
    mustas = cv2.resize(mustas, (mustas_width, mustas_height))

    for i in range(mustas_height):
        for j in range(mustas_width):
            for k in range(3):
                if mustas[i][j][k] < 235:
                    img[y + i - int(-0.6 * face_height)][x + j][k] = mustas[i][j][k] * (mustas[i][j][3]/255) + img[y + i - int(-0.6 * face_height)][x + j][k] * (1 - mustas[i][j][3]/255)
    return img

In [ ]:
import pathlib
import cv2
cascade_path = pathlib.Path(cv2.__file__).parent.absolute() / "data/haarcascade_frontalface_default.xml"
clf = cv2.CascadeClassifier(str(cascade_path))

imgname = "./img/mustas.png"
image = cv2.imread(imgname, cv2.IMREAD_UNCHANGED)

# Borrowed from: https://www.youtube.com/watch?v=5cg_yggtkso&ab_channel=NeuralNine
camera = cv2.VideoCapture(0)

while True:
    _, frame = camera.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = clf.detectMultiScale(
        gray, 
        scaleFactor = 1.1,
        minNeighbors = 10,
        minSize = (100, 100),
        flags = cv2.CASCADE_SCALE_IMAGE
    )

    for(x, y, w, h) in faces:
        frame = put_mustas(image, frame, x, y, w, h)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 255, 0), 2)
    
    cv2.imshow("Face", frame)
    c = cv2.waitKey(1)
    if c == 27:
        break

camera.release()
cv2.destroyAllWindows()